In [51]:
from pathlib import Path

import pandas as pd
import numpy as np

import altair as alt
from altair_saver import save

from IPython.display import Markdown, display


In [7]:
dataPath = Path("../data/raw/")
interimDataPath = Path("../data/interim/")

def loadData():
    df = pd.read_csv(dataPath / "atlanticInterpolated.csv")
    z = df.z.values
    dropColumns = []
    for col in df.columns:
        if df.loc[30, col] > df.loc[0, col] - 0.1:
            dropColumns.append(col)
    df = df.drop(columns=dropColumns)
    # df = pd.concat([df for _ in range(100)], axis=1)
    temps = df.iloc[:, 1:].values
    surfaceTemps = temps[:2, :].mean(axis=0)
    return df, temps, z, surfaceTemps,

In [8]:
print("Load data")
df, temps, z, surfaceTemps = loadData()
print("Data loaded")

Load data
Data loaded


In [9]:
thresholdTemperatureDifference = 0.1

In [41]:
prof = "2494:May:2010"
longDf = pd.melt(df.query('z>-100').loc[:,['z',prof]],id_vars=['z'])
longDf = longDf.rename(columns={'variable':'profile', 'value':'Temperature'})
longDf.head()

,z,profile,Temperature
0,-3.0,2494:May:2010,11.69600
1,-6.0,2494:May:2010,11.69726
2,-9.0,2494:May:2010,11.69900
3,-12.0,2494:May:2010,11.69900
4,-15.0,2494:May:2010,11.69900


In [55]:
fs = 18
tempProfile = alt.Chart(longDf,
                        title='Example Temperature Profile with Mixed Layer Depth',
                        width=400,
                        height=700,
                       ).mark_line().encode(
x='Temperature:Q',
y='z:Q')
mldDot = alt.Chart(pd.DataFrame({'z':[-48],'Temperature':[11.683]})).mark_circle(color='red',size=200).encode(
    x='Temperature:Q',
    y='z:Q')
layers = (tempProfile + mldDot).configure_axis(labelFontSize=fs,titleFontSize=fs).configure_title(fontSize=fs)
# save(layers,'test.svg')
# layers


In [64]:
def mixedLayerIndex(temperature:np.ndarray,thresholdTemperatureDifference:float):
    surfaceTemps = temperature[:2].mean()
    depthIndex = 2
    temperatureDifference = surfaceTemps - temperature[depthIndex]
    while temperatureDifference < thresholdTemperatureDifference:
        depthIndex += 1
        temperatureDifference = surfaceTemps - temperature[depthIndex]
    return depthIndex
        
temperature = np.array([5.0,5.0,4.95,4.89,4.85])
targetDepthIndex = 3
output = mixedLayerIndex(temperature=temperature,thresholdTemperatureDifference=0.1)
assert output == targetDepthIndex,f"output:{output},targetDepthIndex:{targetDepthIndex}"

temperature = np.array([5.0,5.0,4.95,4.94,4.93])
targetDepthIndex = 3
output = mixedLayerIndex(temperature=temperature,thresholdTemperatureDifference=0.1)
assert output == targetDepthIndex,f"output:{output},targetDepthIndex:{targetDepthIndex}"


IndexError: index 5 is out of bounds for axis 0 with size 5

In [67]:
def mixedLayerIndex(temperature:np.ndarray,thresholdTemperatureDifference:float):
    surfaceTemps = temperature[:2].mean()
    depthIndex = 2
    temperatureDifference = surfaceTemps - temperature[depthIndex]
    while (temperatureDifference < thresholdTemperatureDifference) and (depthIndex < temperature.shape[0]-1):
        depthIndex += 1
        temperatureDifference = surfaceTemps - temperature[depthIndex]
    return depthIndex
temperature = np.array([5.0,5.0,4.95,4.94,4.93])
targetDepthIndex = 4
output = mixedLayerIndex(temperature=temperature,thresholdTemperatureDifference=0.1)
assert output == targetDepthIndex,f"output:{output},targetDepthIndex:{targetDepthIndex}"


In [78]:
def mixedLayerIndexArray(temperature:np.ndarray,thresholdTemperatureDifference:float):
    surfaceTemps = temperature[:2].mean(axis=0)
    depthIndexList = []
    for col in range(temperature.shape[1]):
        depthIndex = 2
        temperatureDifference = surfaceTemps[col] - temperature[depthIndex,col]
        while (temperatureDifference < thresholdTemperatureDifference) and (depthIndex < temperature.shape[0]-1):
            depthIndex += 1
            temperatureDifference = surfaceTemps[col] - temperature[depthIndex,col]
        depthIndexList.append(depthIndex)
    depthIndexArray = np.array(depthIndexList)
    return depthIndexArray
temperature = np.array([
    [5.0,5.0,4.95,4.89,4.85],
    [5.0,5.0,4.95,4.94,4.93]
]).T
print(f"Shape of temperature array: {temperature.shape}")
assert temperature.shape[1] == 2
targetDepthIndexArray = np.array([3,4])
output = mixedLayerIndexArray(temperature=temperature,thresholdTemperatureDifference=0.1)
np.testing.assert_array_equal(output,targetDepthIndexArray)


Shape of temperature array: (5, 2)


In [83]:
def mixedLayerIndexDataframe(temperatureDf:pd.DataFrame,thresholdTemperatureDifference:float):
    surfaceTemps = temperatureDf.iloc[:2].mean(axis=0)
    depthIndexList = []
    baseMixedLayerTemperature = []
    for col in temperatureDf.columns:
        depthIndex = 2
        temperatureDifference = surfaceTemps.iloc[col] - temperatureDf.iloc[depthIndex].loc[col]
        while (temperatureDifference < thresholdTemperatureDifference) and (depthIndex < temperature.shape[0]-1):
            depthIndex += 1
            temperatureDifference = surfaceTemps.iloc[col] - temperatureDf.iloc[depthIndex].loc[col]
        depthIndexList.append(depthIndex)
        baseMixedLayerTemperature.append(temperatureDf.iloc[depthIndex].loc[col])

    mixedLayerDf = pd.DataFrame({'depthIndex': depthIndexList,'mlTemp':baseMixedLayerTemperature})
    return mixedLayerDf

temperature = np.array([
    [5.0,5.0,4.95,4.89,4.85],
    [5.0,5.0,4.95,4.94,4.93]
]).T
temperatureDf = pd.DataFrame(temperature,columns=["profA","profB"])
print(f"Shape of temperature array: {temperatureDf.shape}")
assert temperatureDf.shape[1] == 2
targetMixedLayerDf = pd.DataFrame({'depthIndex':[3,4],'mlTemp':[4.89,4.93]})
outputDf = mixedLayerIndexDataframe(temperatureDf=temperatureDf,thresholdTemperatureDifference=0.1)
pd.testing.assert_frame_equal(outputDf,targetMixedLayerDf)


Shape of temperature array: (5, 2)


TypeError: Cannot index by location index with a non-integer key

In [ ]:
pd.testing.

In [84]:
    surfaceTemps = temperatureDf.iloc[:2].mean(axis=0)
    depthIndexList = []
    baseMixedLayerTemperature = []
    for col in temperatureDf.columns:
        depthIndex = 2
        temperatureDifference = surfaceTemps.iloc[col] - temperatureDf.iloc[depthIndex].loc[col]
        while (temperatureDifference < thresholdTemperatureDifference) and (depthIndex < temperature.shape[0]-1):
            depthIndex += 1
            temperatureDifference = surfaceTemps.iloc[col] - temperatureDf.iloc[depthIndex].loc[col]
        depthIndexList.append(depthIndex)
        baseMixedLayerTemperature.append(temperatureDf.iloc[depthIndex].loc[col])

    mixedLayerDf = pd.DataFrame({'depthIndex': depthIndexList,'mlTemp':baseMixedLayerTemperature})


TypeError: Cannot index by location index with a non-integer key

In [ ]:
def numpyNonZeroColumnWise(
    temps: np.ndarray,
    surfaceTemps: np.ndarray,
    thresholdTemperatureDifference: float,
    z: np.ndarray,
):
    condition = temps < (surfaceTemps - thresholdTemperatureDifference)
    mldIndex = np.array(
        [condition[:, colIdx].nonzero()[0][0] for colIdx in range(temps.shape[1])]
    )
    mldDepth = np.array([z[idx] for idx in mldIndex])[:, np.newaxis]
    mldDepth = z[mldIndex][:, np.newaxis]
    mldTemp = np.array([temps[idx, colIdx] for colIdx, idx in enumerate(mldIndex)])[
        :, np.newaxis
    ]
    return mldDepth, mldTemp


mldnumpyNonZeroColumnWise, numpymldTempNonZeroColumnWise = numpyNonZeroColumnWise(
    temps=temps, surfaceTemps=surfaceTemps, thresholdTemperatureDifference=0.1, z=z
)

In [ ]:
def createVisualisationDataframes(df:pd.DataFrame,mlDepth:np.ndarray,mldTemp:np.ndarray):
    meltDf = pd.melt(
        pd.concat([
            df.iloc[:,[0]],
            df.iloc[:,1:].sample(10,axis=1).drop(columns=['z'],errors='ignore')],axis=1),id_vars=['z'])
    mldDf = pd.DataFrame(np.hstack([mlDepth,mldTemp]),index=df.columns[1:],columns=["mld","mldTemp"]).reset_index()
    return meltDf,mldDf
meltDf,mldDf = createVisualisationDataframes(df=df,mlDepth=mldnumpyNonZeroColumnWise,mldTemp=numpymldTempNonZeroColumnWise)